In [2]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from tqdm import tqdm
smoothie = SmoothingFunction().method4
rouge = evaluate.load('rouge')
messages_final = json.load(open('data/messages_final.json'))
messages_boh_original_final = json.load(open('data/messages_boh_original_final.json'))
messages_no_sw_original_final = json.load(open('data/messages_no_sw_original_final.json'))
test_data = json.load(open('data/test_data.json'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/data/sunilruf/llama/fp16_no/rank_32_1/checkpoint-50/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")

In [4]:
i = 1
bleu_2_final, bleu_2_boh_original_final, bleu_2_no_sw_original_final = [], [], []
bleu_4_final, bleu_4_boh_original_final, bleu_4_no_sw_original_final = [], [], []
bert_score_final, bert_score_boh_original_final, bert_score_no_sw_original_final = [], [], []
rouge_final, rouge_boh_original_final, rouge_no_sw_original_final = [], [], []
meteor_final, meteor_boh_original_final, meteor_no_sw_original_final = [], [], []

for j in tqdm(range(100)):
    
    for i in range(1, len(test_data[j]),2):
        #print(i)
        if messages_final[j][i]['content'] == '' or test_data[j][i]['content'] == '' or messages_boh_original_final[j][i]['content'] == '' or messages_no_sw_original_final[j][i]['content'] == '':
            continue
        #print(messages_final[j][i]['content'])
        
        bleu_2_final.append(sentence_bleu(messages_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_2_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_2_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_4_final.append(sentence_bleu(messages_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bleu_4_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bleu_4_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bert_score_final.append(score([messages_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        bert_score_boh_original_final.append(score([messages_boh_original_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        bert_score_no_sw_original_final.append(score([messages_no_sw_original_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        rouge_final.append(rouge.compute(predictions=[messages_final[j][i]['content']], references=[test_data[j][i]['content']]))
        rouge_boh_original_final.append(rouge.compute(predictions=[messages_boh_original_final[j][i]['content']], references=[test_data[j][i]['content']]))
        rouge_no_sw_original_final.append(rouge.compute(predictions=[messages_no_sw_original_final[j][i]['content']], references=[test_data[j][i]['content']]))
        meteor_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        meteor_boh_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_boh_original_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        meteor_no_sw_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_no_sw_original_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        
        

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [20:20<00:00, 12.21s/it]


In [5]:
rougel_final = [i['rougeL'] for i in rouge_final]
rougel_boh_original_final = [i['rougeL'] for i in rouge_boh_original_final]
rougel_no_sw_original_final = [i['rougeL'] for i in rouge_no_sw_original_final]

In [6]:
bert_p_boh = [i[0].item() for i in bert_score_boh_original_final]
bert_r_boh = [i[1].item() for i in bert_score_boh_original_final]
bert_f1_boh = [i[2].item() for i in bert_score_boh_original_final]

bert_p_no_sw = [i[0].item() for i in bert_score_no_sw_original_final]
bert_r_no_sw = [i[1].item() for i in bert_score_no_sw_original_final]
bert_f1_no_sw = [i[2].item() for i in bert_score_no_sw_original_final]

bert_p = [i[0].item() for i in bert_score_final]
bert_r = [i[1].item() for i in bert_score_final]
bert_f1 = [i[2].item() for i in bert_score_final]

In [7]:
import statistics

print("Bleu 2 Final: ", statistics.mean(bleu_2_final))
print("Bleu 2 Boh Original Final: ", statistics.mean(bleu_2_boh_original_final))
print("Bleu 2 No Sw Original Final: ", statistics.mean(bleu_2_no_sw_original_final))
print("Bleu 4 Final: ", statistics.mean(bleu_4_final))
print("Bleu 4 Boh Original Final: ", statistics.mean(bleu_4_boh_original_final))
print("Bleu 4 No Sw Original Final: ", statistics.mean(bleu_4_no_sw_original_final))
print("Rouge L Final: ", statistics.mean(rougel_final))
print("Rouge L Boh Original Final: ", statistics.mean(rougel_boh_original_final))
print("Rouge L No Sw Original Final: ", statistics.mean(rougel_no_sw_original_final))
print("Bert Precision Final: ", statistics.mean(bert_p))
print("Bert Recall Final: ", statistics.mean(bert_r))
print("Bert F1 Final: ", statistics.mean(bert_f1))
print("Bert Precision Boh Original Final: ", statistics.mean(bert_p_boh))
print("Bert Recall Boh Original Final: ", statistics.mean(bert_r_boh))
print("Bert F1 Boh Original Final: ", statistics.mean(bert_f1_boh))
print("Bert Precision No Sw Original Final: ", statistics.mean(bert_p_no_sw))
print("Bert Recall No Sw Original Final: ", statistics.mean(bert_r_no_sw))
print("Bert F1 No Sw Original Final: ", statistics.mean(bert_f1_no_sw))
print("Meteor Final: ", statistics.mean(meteor_final))
print("Meteor Boh Original Final: ", statistics.mean(meteor_boh_original_final))
print("Meteor No Sw Original Final: ", statistics.mean(meteor_no_sw_original_final))


Bleu 2 Final:  0.022428202955559518
Bleu 2 Boh Original Final:  0.021479569843545387
Bleu 2 No Sw Original Final:  0.021154077448443737
Bleu 4 Final:  0.005236413654820756
Bleu 4 Boh Original Final:  0.00512813465055343
Bleu 4 No Sw Original Final:  0.005084864787179422
Rouge L Final:  0.21916091051851186
Rouge L Boh Original Final:  0.15947136262482073
Rouge L No Sw Original Final:  0.13094655334062932
Bert Precision Final:  0.5729715586562929
Bert Recall Final:  0.5474313054122893
Bert F1 Final:  0.5559400214085637
Bert Precision Boh Original Final:  0.4919859349333333
Bert Recall Boh Original Final:  0.4395153865037443
Bert F1 Boh Original Final:  0.46017099744429796
Bert Precision No Sw Original Final:  0.5137757430923025
Bert Recall No Sw Original Final:  0.43972462767277165
Bert F1 No Sw Original Final:  0.4688320349339219
Meteor Final:  0.2794655716150985
Meteor Boh Original Final:  0.20115607084670775
Meteor No Sw Original Final:  0.21978682335195493


In [8]:
import json
import torch
from tqdm import tqdm
import statistics
messages_boh = json.load(open('data/messages_boh_final.json'))
messages_sw = json.load(open('data/messages_no_sw_final.json'))
messages_final = json.load(open('data/messages_final.json'))

In [13]:
boh_length, sw_length, final_length = [], [], []
for j in tqdm(range(100)):
    for i in range(0,len(messages_final[j])):
        final_length.append(len(tokenizer.tokenize(messages_final[j][i]['content'])))
        sw_length.append(len(tokenizer.tokenize(messages_sw[j][i]['content'])))
        boh_length.append(len(tokenizer.tokenize(messages_boh[j][i]['content'])))

100%|██████████| 100/100 [00:00<00:00, 299.25it/s]


In [14]:
print("actual length ", statistics.mean(final_length))
print("boh length ", statistics.mean(boh_length))
print("sw length ", statistics.mean(sw_length))

actual length  35.53556034482759
boh length  17.744073275862068
sw length  17.90301724137931


In [15]:
import torch

perplexity_final = torch.load('data/perplexity_final.pt')
perplexity_boh = torch.load('data/perplexity_boh_final.pt')
perplexity_sw = torch.load('data/perplexity_no_sw_final.pt')


In [16]:
perplexity_final1 = []
perplexity_final1.extend(per for sublist in perplexity_final for per in sublist )
perplexity_boh1 = []
perplexity_boh1.extend(per for sublist in perplexity_boh for per in sublist )
perplexity_sw1 = []
perplexity_sw1.extend(per for sublist in perplexity_sw for per in sublist )

In [17]:
print("Perplexity Final: ", torch.mean(torch.tensor(perplexity_final1)))
print("Perplexity Boh Original Final: ", torch.mean(torch.tensor(perplexity_boh1)))
print("Perplexity No Sw Original Final: ", torch.mean(torch.tensor(perplexity_sw1)))

Perplexity Final:  tensor(9.2122)
Perplexity Boh Original Final:  tensor(53.3155)
Perplexity No Sw Original Final:  tensor(34.3808)
